In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [54]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

In [56]:
df=pd.read_csv(r'C:\Users\anshul\Desktop\ML_LLM\ml_llm\nlp\data\Tweets.csv').dropna()
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [57]:
df['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [58]:
cat_id={'neutral':1, 'negative':0, 'positive':2}
df['class']=df['sentiment'].map(cat_id)

In [59]:
BATCH_SIZE = 512
NUM_EPOCHS = 80

In [60]:
x=df['text'].values
y=df['class'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state=123)

In [61]:
one_hot=CountVectorizer()

x_train_onehot=one_hot.fit_transform(x_train)
x_test_onehot=one_hot.transform(x_test)

In [62]:
class sentimentdata(Dataset):
    def __init__(self,x,y):
        super().__init__()
        self.x=torch.tensor(x.toarray())
        self.y=torch.tensor(y).type(torch.LongTensor)
        self.len=len(self.x)

    def __len__(self):
        return self.len
    
    def __getitem__(self,index):
        return self.x[index],self.y[index]
    

    
train_ds=sentimentdata(x=x_train_onehot,y=y_train)
test_ds=sentimentdata(x=x_test_onehot,y=y_test)

In [63]:
train_dataloader=DataLoader(dataset=train_ds,batch_size=BATCH_SIZE, shuffle=True)
test_dataloader=DataLoader(test_ds,batch_size=15000)



In [64]:
    
class SentimentModel(nn.Module):
    def __init__(self, NUM_FEATURES, NUM_CLASSES, HIDDEN = 10):
        super().__init__()
        self.linear = nn.Linear(NUM_FEATURES, HIDDEN)
        self.linear2 = nn.Linear(HIDDEN, NUM_CLASSES)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.log_softmax(x)
        return x
        
model=SentimentModel(NUM_FEATURES=x_train_onehot.shape[1],NUM_CLASSES=3)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters())

In [65]:
train_losses=[]

for e in range(NUM_EPOCHS):
    curr_loss=0
    for x_batch,y_batch in train_dataloader:
        optimizer.zero_grad()
        y_pred_log = model(x_batch)
        loss = criterion(y_pred_log, y_batch.long())
        curr_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_losses.append(curr_loss)
    print(f"Epoch {e}, Loss: {curr_loss}")

RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float

In [ ]:
sns.lineplot(x=list(range(len(train_losses))), y= train_losses)
plt.show()